# MNIST Model Constructor
---
The following is a python script for reproducing the MNIST model I used to classify my images.

## **NOTE**: Due the the non-parallizeable nature of python, this runs extremely slowly. Set aside a couple of hours if you wish to create your own model with the following parameters.

Run the code below to regenerate the same model I used in the drawing tool.

In [ ]:
import sys
scripts_path = r"../"

if scripts_path not in sys.path:
    sys.path.append(scripts_path)

from NeuralNetworkScripts.NeuralNetwork import NeuralNetwork
from NeuralNetworkScripts.Layer import FullyConnectedLayer
from NeuralNetworkScripts.DataHandler import DataHelper
import numpy as np
from tensorflow.keras.datasets import mnist

np.random.seed(100) # unseeded for cv

# Load the MNIST dataset and split labels into one-hot
(digitsTrainSet, digitsTrainLabels), (digitsTestSet, digitsTestLabels) = mnist.load_data()

order = [0,1,2,3,4,5,6,7,8,9]
trainLabels,_ = DataHelper.toOneHot(digitsTrainLabels, order)
testLabels,_ = DataHelper.toOneHot(digitsTestLabels, order)

trainSet = digitsTrainSet.reshape(60000, 28*28) # stretch out to naively get 784 features
testSet = digitsTestSet.reshape(10000, 28*28)

print("Current Dimensions:")
print("Train Set:", trainSet.shape)
print("Train Labels:", trainLabels.shape)
print("Test Set:", testSet.shape)
print("Test Labels:", testLabels.shape)

#trainSet, trainLabels, testSet, testLabels = DataHelper.trainTestSplit(trainSet, trainLabels) # cross-validation used to find hyperparameters

Standardizer = DataHelper.standardizer(trainSet)
trainSet = DataHelper.standardizeCompute(trainSet,Standardizer)
testSet = DataHelper.standardizeCompute(testSet,Standardizer)

x = NeuralNetwork(
                # NOTE: Best found hyperparams after cv
                #inputDropout=x specified here
                #batchSize=default 32 #batchTests
                hidden2 = FullyConnectedLayer(numNodes=400,activation='ReLU',dropout=.05), 
                hidden3 = FullyConnectedLayer(numNodes=400,activation='ReLU',dropout=.05),
                output = FullyConnectedLayer(numNodes=10,activation='softmax')
                )

# specify Adam and AdamW. weight decay means nothing if used with Adam
print("Starting model training. Epochs will periodically print between 10 and 20 minutes.")
x.train(trainSet, trainLabels, epochs=12, learningRate=.001,loss='AdamW',weightDecay=.17) 

lossTraining, trainGuesses = x.test(trainSet, trainLabels)

predictedTrain = np.argmax(trainGuesses, axis=1)
trueTrain = np.argmax(trainLabels, axis=1)
trainAccuracy = np.mean(predictedTrain == trueTrain) * 100

print("\nFinal Training Loss:", lossTraining)
print("Training Acc: {:.2f}%".format(trainAccuracy))

lossTesting, testGuesses = x.evaluate(testSet, testLabels)

predictedTest = np.argmax(testGuesses, axis=1)
trueTest = np.argmax(testLabels, axis=1)
testAccuracy = np.mean(predictedTest == trueTest) * 100

print("\nFinal Testing Loss:", lossTesting)
print("Testing Acc: {:.2f}%".format(testAccuracy))

x.saveModel("MNISTModel")
print("Model saved as MNISTModel within the local directory.")

Current Dimensions:
Train Set: (60000, 784)
Train Labels: (60000, 10)
Test Set: (10000, 784)
Test Labels: (10000, 10)
Starting model training. Epochs will periodically print between 10 and 20 minutes.
loss for epoch:  1 : 2.2796016050843533
loss for epoch:  2 : 0.6604765615479542
loss for epoch:  3 : 0.4167719557732928
loss for epoch:  4 : 0.3257775793987371
loss for epoch:  5 : 0.26738986036664114
loss for epoch:  6 : 0.22127803972857935
loss for epoch:  7 : 0.19084752899748855
loss for epoch:  8 : 0.1595542247352233
loss for epoch:  9 : 0.1309223633227243
loss for epoch:  10 : 0.1109718298406343
loss for epoch:  11 : 0.09110099007333433
loss for epoch:  12 : 0.07953411972331074

Final Training Loss: 0.07175497542480534
Training Acc: 97.88%

Final Testing Loss: 0.1209791430887968
Testing Acc: 96.45%
Model saved as MNISTModel within the local directory.


The following code chunks show the model being reloaded and working to classify the same test set.

In [ ]:
    
savedModel = NeuralNetwork(model="MNISTModel")
lossSaved, guessesSaved = savedModel.evaluate(testSet, testLabels)

predictedSaved = np.argmax(guessesSaved, axis=1)
trueSaved = np.argmax(testLabels, axis=1)
accuracySaved = np.mean(predictedSaved == trueSaved) * 100

print("\nFinal Testing Loss:", lossSaved)
print("Testing Acc: {:.2f}%".format(accuracySaved))


Final Testing Loss: 0.12116224664761685
Testing Acc: 96.59%
